<a href="https://colab.research.google.com/github/OlajideFemi/Carbon-Footprint/blob/main/Queueing_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from datetime import datetime

def fetch_port_daily_data(port_ids=None, countries=None, start_date="2024-04-01", end_date="2026-03-31"):
    """
    Fetch daily port-level data from Daily_Ports_Data

    Parameters:
    - port_ids: List of port IDs (e.g., ['port1325', 'port1198']) or None for all ports
    - countries: List of country names or ISO3 codes to filter
    - start_date: YYYY-MM-DD
    - end_date: YYYY-MM-DD
    """

    base_url = "https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/Daily_Ports_Data/FeatureServer/0/query"

    # Build where clause - using year, month, day fields for efficient filtering
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')

    # Create date range conditions using the separate fields
    date_conditions = []

    # For simplicity, we'll use the string date field which works with comparisons
    date_conditions = [f"date >= '{start_date}'", f"date <= '{end_date}'"]

    # Add port filters if specified
    if port_ids:
        port_list = "','".join(port_ids)
        date_conditions.append(f"portid IN ('{port_list}')")

    # Add country filters if specified
    if countries:
        country_list = "','".join(countries)
        date_conditions.append(f"country IN ('{country_list}') OR ISO3 IN ('{country_list}')")

    where_clause = " AND ".join(date_conditions)

    params = {
        "where": where_clause,
        "outFields": "date,year,month,day,portid,portname,country,ISO3,portcalls_roro,portcalls_container,portcalls_dry_bulk,portcalls_tanker",
        "returnGeometry": "false",
        "resultRecordCount": "5000",
        "maxRecordCountFactor": "5",
        "orderByFields": "portid ASC, date ASC",
        "f": "json"
    }

    print(f"\n📡 Fetching daily port data...")
    print(f"   Period: {start_date} to {end_date}")
    if port_ids:
        print(f"   Ports: {port_ids}")
    if countries:
        print(f"   Countries: {countries}")

    all_features = []
    offset = 0

    while True:
        params["resultOffset"] = str(offset)

        response = requests.get(base_url, params=params)

        if response.status_code != 200:
            print(f"❌ Error: {response.status_code}")
            break

        data = response.json()

        if 'error' in data:
            print(f"❌ API Error: {data['error'].get('message')}")
            break

        features = data.get('features', [])
        if not features:
            break

        all_features.extend(features)
        print(f"   Retrieved {len(all_features)} records so far...")

        if len(features) < 5000:  # Last page
            break

        offset += 5000

    if not all_features:
        print("❌ No data returned")
        return pd.DataFrame()

    # Convert to DataFrame
    records = []
    for f in all_features:
        attrs = f['attributes']
        # Convert date string to datetime
        if 'date' in attrs:
            attrs['date'] = pd.to_datetime(attrs['date'])
        records.append(attrs)

    df = pd.DataFrame(records)
    print(f"\n✅ Retrieved {len(df)} total records")

    return df

# Step 1: Get all ports to find UK and French port IDs
print("="*80)
print("STEP 1: Finding UK and French Port IDs")
print("="*80)

ports_url = "https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/PortWatch_ports_database/FeatureServer/0/query"
params = {
    "where": "country IN ('United Kingdom','France','Netherlands','Belgium')",
    "outFields": "portid,portname,country,ISO3,lat,lon",
    "returnGeometry": "false",
    "f": "json"
}

response = requests.get(ports_url, params=params)
ports_data = response.json()

europe_ports = []
for feat in ports_data.get('features', []):
    attrs = feat['attributes']
    europe_ports.append({
        'portid': attrs.get('portid'),
        'portname': attrs.get('portname'),
        'country': attrs.get('country'),
        'ISO3': attrs.get('ISO3')
    })

ports_df = pd.DataFrame(europe_ports)
print(f"\nFound {len(ports_df)} ports in UK, France, Netherlands, Belgium")
print("\nKey ports:")
key_port_names = ['Dover', 'Calais', 'Rotterdam', 'Zeebrugge', 'Antwerp', 'Felixstowe', 'Southampton']
key_ports = ports_df[ports_df['portname'].str.contains('|'.join(key_port_names), case=False, na=False)]
print(key_ports.to_string())

# Save port IDs for later
key_ports.to_csv('key_europe_ports.csv', index=False)
port_ids_list = key_ports['portid'].tolist()

# Step 2: Fetch FY24/25 daily data for these ports
print("\n" + "="*80)
print("STEP 2: Fetching FY24/25 Daily Port Data")
print("="*80)

daily_data = fetch_port_daily_data(
    port_ids=port_ids_list,
    start_date="2024-04-01",
    end_date="2026-03-31"
)

if not daily_data.empty:
    # Save master file
    daily_data.to_csv('fy24_25_port_daily_data.csv', index=False)
    print("\n💾 Saved to 'fy24_25_port_daily_data.csv'")

    # Summary statistics
    print("\n" + "="*80)
    print("📊 FY24/25 DATA SUMMARY")
    print("="*80)

    summary = daily_data.groupby(['portname', 'country']).agg({
        'portcalls_roro': ['sum', 'mean', 'count'],
        'date': ['min', 'max']
    }).round(2)

    print(summary)

    # Daily time series for key ports
    for port in daily_data['portname'].unique():
        port_df = daily_data[daily_data['portname'] == port].sort_values('date')
        print(f"\n📈 {port} daily RoRo calls:")
        print(port_df[['date', 'portcalls_roro']].head(10))
        print(f"... and {len(port_df)-10} more days")

STEP 1: Finding UK and French Port IDs

Found 114 ports in UK, France, Netherlands, Belgium

Key ports:
      portid     portname         country ISO3
5    port206       Calais          France  FRA
21   port301        Dover  United Kingdom  GBR
24   port343   Felixstowe  United Kingdom  GBR
33    port57      Antwerp         Belgium  BEL
37  port1424    Zeebrugge         Belgium  BEL
60  port1216  Southampton  United Kingdom  GBR

STEP 2: Fetching FY24/25 Daily Port Data

📡 Fetching daily port data...
   Period: 2024-04-01 to 2026-03-31
   Ports: ['port206', 'port301', 'port343', 'port57', 'port1424', 'port1216']
   Retrieved 4104 records so far...

✅ Retrieved 4104 total records

💾 Saved to 'fy24_25_port_daily_data.csv'

📊 FY24/25 DATA SUMMARY
                           portcalls_roro                   date           
                                      sum  mean count        min        max
portname    country                                                        
Antwerp     Belgiu

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Load your data
df = pd.read_csv('fy24_25_port_daily_data.csv')
df['date'] = pd.to_datetime(df['date'])

class PortDisruptionModel:
    """Queueing model for port disruption scenarios using actual port data"""

    def __init__(self, df):
        self.df = df
        self.ports = df['portname'].unique()
        print(f"Loaded data for {len(self.ports)} ports: {', '.join(self.ports)}")

    def analyze_port(self, port_name):
        """Calculate baseline statistics for a specific port"""
        port_df = self.df[self.df['portname'] == port_name].copy()
        port_df = port_df.sort_values('date')

        # Basic statistics
        stats = {
            'port': port_name,
            'country': port_df['country'].iloc[0],
            'days': len(port_df),
            'total_calls': port_df['portcalls_roro'].sum(),
            'mean_calls': port_df['portcalls_roro'].mean(),
            'std_calls': port_df['portcalls_roro'].std(),
            'max_calls': port_df['portcalls_roro'].max(),
            'zero_days': (port_df['portcalls_roro'] == 0).sum(),
            'pct_zero': (port_df['portcalls_roro'] == 0).mean() * 100
        }

        # Weekly pattern
        port_df['day_of_week'] = port_df['date'].dt.dayofweek
        weekly = port_df.groupby('day_of_week')['portcalls_roro'].agg(['mean', 'std'])

        return stats, weekly, port_df

    def simulate_disruption(self, port_name, disruption_date, duration_days,
                           capacity_reduction_pct, use_historical=True):
        """
        Simulate queue buildup during disruption

        Parameters:
        - port_name: Name of port
        - disruption_date: Start date (YYYY-MM-DD)
        - duration_days: How long disruption lasts
        - capacity_reduction_pct: % reduction in processing capacity
        - use_historical: If True, use actual arrival patterns; if False, use average
        """

        # Get port data
        port_df = self.df[self.df['portname'] == port_name].copy()
        port_df = port_df.sort_values('date')

        # Calculate baseline (use 60 days before disruption)
        baseline_end = pd.to_datetime(disruption_date) - timedelta(days=1)
        baseline_start = baseline_end - timedelta(days=60)
        baseline = port_df[
            (port_df['date'] >= baseline_start) &
            (port_df['date'] <= baseline_end)
        ]

        if len(baseline) < 30:
            print(f"Warning: Only {len(baseline)} days in baseline, using all available data")
            baseline = port_df[port_df['date'] < pd.to_datetime(disruption_date)]

        # Calculate arrival rate (λ) from baseline
        lambda_mean = baseline['portcalls_roro'].mean()
        lambda_std = baseline['portcalls_roro'].std()

        # Get processing rate (μ) - assume normal capacity can handle peak + 20%
        peak_day = baseline['portcalls_roro'].max()
        mu_normal = peak_day * 1.2  # 20% spare capacity

        # Disrupted processing rate
        mu_disrupted = mu_normal * (1 - capacity_reduction_pct/100)

        print(f"\n{'='*60}")
        print(f"DISRUPTION SIMULATION: {port_name}")
        print('='*60)
        print(f"Baseline period: {baseline_start.date()} to {baseline_end.date()}")
        print(f"Normal arrival rate (λ): {lambda_mean:.2f} calls/day (σ={lambda_std:.2f})")
        print(f"Normal capacity (μ): {mu_normal:.2f} calls/day")
        print(f"Disrupted capacity ({capacity_reduction_pct}% reduction): {mu_disrupted:.2f} calls/day")

        # Run simulation
        simulation_days = duration_days + 30  # Include recovery period
        sim_dates = [pd.to_datetime(disruption_date) + timedelta(days=i)
                    for i in range(simulation_days)]

        # Use actual arrivals if available, otherwise use random from baseline distribution
        queue = 0
        results = []

        for i, sim_date in enumerate(sim_dates):
            # Get actual arrival if in historical data
            hist_row = port_df[port_df['date'] == sim_date]

            if use_historical and not hist_row.empty:
                arrivals = hist_row['portcalls_roro'].iloc[0]
            else:
                # Simulate arrivals using baseline distribution
                arrivals = max(0, np.random.normal(lambda_mean, lambda_std))

            # Determine processing capacity
            if i < duration_days:
                # During disruption
                capacity = mu_disrupted
            else:
                # Recovery period - capacity returns to normal
                capacity = mu_normal

            # Queueing logic
            if queue > 0:
                # Process queue first
                processed = min(capacity, queue + arrivals)
                queue = queue + arrivals - processed
            else:
                # No queue, process what we can
                if arrivals <= capacity:
                    queue = 0
                else:
                    queue = arrivals - capacity

            # Calculate wait time
            if queue > 0 and capacity > 0:
                wait_time_hours = (queue / capacity) * 24
            else:
                wait_time_hours = 0

            results.append({
                'date': sim_date,
                'day': i,
                'arrivals': arrivals,
                'capacity': capacity,
                'queue': queue,
                'wait_time_hours': wait_time_hours,
                'phase': 'disruption' if i < duration_days else 'recovery'
            })

        results_df = pd.DataFrame(results)

        # Summary statistics
        peak_queue = results_df['queue'].max()
        peak_wait = results_df['wait_time_hours'].max()
        total_disrupted = results_df[results_df['phase'] == 'disruption']['arrivals'].sum()
        total_processed = (results_df[results_df['phase'] == 'disruption']['capacity'].sum() *
                          results_df[results_df['phase'] == 'disruption']['day'].count())

        print(f"\n📈 SIMULATION RESULTS:")
        print(f"   Peak queue: {peak_queue:.0f} vessels")
        print(f"   Peak wait time: {peak_wait:.1f} hours")
        print(f"   Total arrivals during disruption: {total_disrupted:.0f}")
        print(f"   Total processing capacity during disruption: {total_processed:.0f}")

        if total_disrupted > total_processed:
            backlog = total_disrupted - total_processed
            print(f"   ⚠ BACKLOG: {backlog:.0f} vessels remain after disruption")
            print(f"   Recovery time: {results_df[results_df['queue']==0].index[0] - duration_days + 1:.0f} days")

        return results_df

    def run_scenarios(self, port_name):
        """Run multiple disruption scenarios for a port"""

        scenarios = [
            {'name': 'Minor (10% reduction, 3 days)', 'reduction': 10, 'duration': 3},
            {'name': 'Moderate (25% reduction, 7 days)', 'reduction': 25, 'duration': 7},
            {'name': 'Major (50% reduction, 14 days)', 'reduction': 50, 'duration': 14},
            {'name': 'Severe (75% reduction, 5 days)', 'reduction': 75, 'duration': 5},
        ]

        # Find a typical week in summer 2025
        summer_data = self.df[
            (self.df['portname'] == port_name) &
            (self.df['date'] >= '2025-06-01') &
            (self.df['date'] <= '2025-08-31')
        ]

        if not summer_data.empty:
            # Pick a random Tuesday in summer (typical busy day)
            tuesdays = summer_data[summer_data['date'].dt.dayofweek == 1]
            if not tuesdays.empty:
                disruption_date = tuesdays.iloc[len(tuesdays)//2]['date']
            else:
                disruption_date = '2025-07-01'
        else:
            disruption_date = '2025-07-01'

        print(f"\n{'='*60}")
        print(f"RUNNING SCENARIOS FOR {port_name}")
        print(f"Disruption start date: {disruption_date}")
        print('='*60)

        results = {}
        for scenario in scenarios:
            print(f"\n▶ Scenario: {scenario['name']}")
            sim_df = self.simulate_disruption(
                port_name,
                disruption_date,
                scenario['duration'],
                scenario['reduction']
            )
            results[scenario['name']] = sim_df

        return results

# Initialize model
model = PortDisruptionModel(df)

# 1. Analyze each port's baseline
print("\n" + "="*60)
print("PORT BASELINE STATISTICS")
print("="*60)

for port in model.ports:
    stats, weekly, _ = model.analyze_port(port)
    print(f"\n📊 {port} ({stats['country']}):")
    print(f"   Daily RoRo calls: mean={stats['mean_calls']:.2f}, std={stats['std_calls']:.2f}")
    print(f"   Total calls (677 days): {stats['total_calls']:.0f}")
    print(f"   Zero days: {stats['zero_days']} ({stats['pct_zero']:.1f}%)")
    print(f"   Weekly pattern:")
    for dow in range(7):
        day_name = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'][dow]
        mean = weekly.loc[dow, 'mean'] if dow in weekly.index else 0
        print(f"     {day_name}: {mean:.2f}")

# 2. Run scenarios for Zeebrugge (busiest RoRo port)
zeebrugge_scenarios = model.run_scenarios('Zeebrugge')

# 3. Run scenarios for Southampton
southampton_scenarios = model.run_scenarios('Southampton')

# 4. Save all simulation results
for port in model.ports:
    port_df = df[df['portname'] == port].copy()
    port_df.to_csv(f'{port.lower()}_daily_data.csv', index=False)

print("\n✅ All data saved to CSV files")

Loaded data for 6 ports: Southampton, Zeebrugge, Calais, Dover, Felixstowe, Antwerp

PORT BASELINE STATISTICS

📊 Southampton (United Kingdom):
   Daily RoRo calls: mean=1.80, std=1.20
   Total calls (677 days): 1229
   Zero days: 95 (13.9%)
   Weekly pattern:
     Mon: 1.83
     Tue: 1.64
     Wed: 1.82
     Thu: 1.87
     Fri: 1.58
     Sat: 1.84
     Sun: 2.01

📊 Zeebrugge (Belgium):
   Daily RoRo calls: mean=8.46, std=2.68
   Total calls (677 days): 5787
   Zero days: 1 (0.1%)
   Weekly pattern:
     Mon: 6.98
     Tue: 9.87
     Wed: 7.62
     Thu: 9.71
     Fri: 8.87
     Sat: 8.48
     Sun: 7.68

📊 Calais (France):
   Daily RoRo calls: mean=0.10, std=0.30
   Total calls (677 days): 67
   Zero days: 617 (90.2%)
   Weekly pattern:
     Mon: 0.07
     Tue: 0.13
     Wed: 0.02
     Thu: 0.17
     Fri: 0.07
     Sat: 0.10
     Sun: 0.11

📊 Dover (United Kingdom):
   Daily RoRo calls: mean=0.00, std=0.04
   Total calls (677 days): 1
   Zero days: 683 (99.9%)
   Weekly pattern:
     Mon

# New section

In [3]:
print("\n" + "="*60)
print("RUNNING SCENARIOS FOR Antwerp")
print("="*60)

antwerp_scenarios = model.run_scenarios('Antwerp')

# Extract and display the results in a clean table
print("\n" + "="*60)
print("ANTWERP SIMULATION RESULTS SUMMARY")
print("="*60)

scenario_results = []
for scenario_name, sim_df in antwerp_scenarios.items():
    # Extract key metrics
    peak_queue = sim_df['queue'].max()
    peak_wait = sim_df['wait_time_hours'].max()
    total_arrivals = sim_df[sim_df['phase'] == 'disruption']['arrivals'].sum()
    total_capacity_during_disruption = sim_df[sim_df['phase'] == 'disruption']['capacity'].sum()
    disruption_days = len(sim_df[sim_df['phase'] == 'disruption'])

    # Calculate if backlog occurred
    # The total processing capacity should be multiplied by the number of days
    backlog = max(0, total_arrivals - (total_capacity_during_disruption))

    scenario_results.append({
        'Scenario': scenario_name,
        'Peak Queue': f"{peak_queue:.0f} vessels",
        'Peak Wait': f"{peak_wait:.1f} hours",
        'Space Needed': f"{peak_queue * 15:.0f} metres", # Assuming 15 meters per vessel
        'Backlog': f"{backlog:.0f} vessels" if backlog > 0 else "None"
    })

# Display as table
results_df = pd.DataFrame(scenario_results)
print(results_df.to_string(index=False))

# Detailed breakdown for each scenario
for scenario_name, sim_df in antwerp_scenarios.items():
    print(f"\n📊 {scenario_name} DETAILS:")
    disruption_days = len(sim_df[sim_df['phase'] == 'disruption'])
    print(f"  Disruption duration: {disruption_days} days")
    print(f"  Daily arrivals during disruption: {sim_df[sim_df['phase'] == 'disruption']['arrivals'].mean():.2f}")
    print(f"  Daily capacity during disruption: {sim_df[sim_df['phase'] == 'disruption']['capacity'].mean():.2f}")
    print(f"  Queue timeline:")

    # Show queue on key days (e.g., every 5 days or at specific points)
    # Adjusting for potentially shorter dataframes to avoid errors
    step = max(1, len(sim_df)//5)
    for day_idx in range(0, len(sim_df), step):
        row = sim_df.iloc[day_idx]
        print(f"    Day {row['day']:.0f}: queue={row['queue']:.0f} vessels, wait={row['wait_time_hours']:.1f}h")


RUNNING SCENARIOS FOR Antwerp

RUNNING SCENARIOS FOR Antwerp
Disruption start date: 2025-07-15 00:00:00

▶ Scenario: Minor (10% reduction, 3 days)

DISRUPTION SIMULATION: Antwerp
Baseline period: 2025-05-15 to 2025-07-14
Normal arrival rate (λ): 2.67 calls/day (σ=1.48)
Normal capacity (μ): 8.40 calls/day
Disrupted capacity (10% reduction): 7.56 calls/day

📈 SIMULATION RESULTS:
   Peak queue: 0 vessels
   Peak wait time: 0.0 hours
   Total arrivals during disruption: 6
   Total processing capacity during disruption: 68

▶ Scenario: Moderate (25% reduction, 7 days)

DISRUPTION SIMULATION: Antwerp
Baseline period: 2025-05-15 to 2025-07-14
Normal arrival rate (λ): 2.67 calls/day (σ=1.48)
Normal capacity (μ): 8.40 calls/day
Disrupted capacity (25% reduction): 6.30 calls/day

📈 SIMULATION RESULTS:
   Peak queue: 0 vessels
   Peak wait time: 0.0 hours
   Total arrivals during disruption: 13
   Total processing capacity during disruption: 309

▶ Scenario: Major (50% reduction, 14 days)

DISRU